### 多线程编程

In [1]:
import threading 
import time


In [3]:
## 查看当前线程的名称
print(threading.current_thread().name)
print(f'当前线程数：{threading.active_count()}')


MainThread
当前线程数：6


In [4]:
# 打印所有活跃线程的详细信息
for thread in threading.enumerate():
    print(f"线程名: {thread.name} | ID: {thread.ident} | 是否存活: {thread.is_alive()}")


线程名: MainThread | ID: 18544 | 是否存活: True
线程名: IOPub | ID: 29200 | 是否存活: True
线程名: Heartbeat | ID: 8992 | 是否存活: True
线程名: Control | ID: 7532 | 是否存活: True
线程名: IPythonHistorySavingThread | ID: 10780 | 是否存活: True
线程名: Thread-3 | ID: 18176 | 是否存活: True


#### 创建线程

In [15]:
def task():
    time.sleep(2)
    print('threading is running')

task() 


threading is running


In [19]:
## 创建线程，运行线程，等待线程结束
t1 = threading.Thread(target=task, name='Thread-1')
t1.start()  ## 启动线程
print(f'当前总线程数：{threading.active_count()}') ## 查看线程数
t1.join()  ## 等待线程结束
print(f'线程是否存活：{t1.is_alive()}') 
print(f'线程名称：{t1.name}')  ## 获取线程名称
print(f'线程ID:{t1.ident}')  ## 获取线程ID


当前总线程数：7
threading is running
线程是否存活：False
线程名称：Thread-1
线程ID:18188


#### 单线程 vs. 多线程

In [20]:
def task(n):
    print(f"任务 {n} 开始")
    time.sleep(2)   ## 模拟I/O等待
    print(f"任务 {n} 完成")
    return n * n


In [21]:
start = time.time()
# 顺序执行5个任务
results = [task(i+1) for i in range(5)]
end = time.time()
print(f"顺序执行结果: {results}")
print(f"总耗时: {end-start:.2f}秒")



任务 1 开始
任务 1 完成
任务 2 开始
任务 2 完成
任务 3 开始
任务 3 完成
任务 4 开始
任务 4 完成
任务 5 开始
任务 5 完成
顺序执行结果: [1, 4, 9, 16, 25]
总耗时: 10.05秒


In [24]:
def main():
    start = time.time()
    threads = []
    for i in range(5):
        t = threading.Thread(target=task, name=f'Thread-{i+1}', args=(i+1,))
        threads.append(t)
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    end = time.time()
    # print(f"结果: {results}")
    print(f"线程池耗时: {end-start:.2f}秒")

if __name__ == "__main__":
    main()


任务 1 开始
任务 2 开始
任务 3 开始
任务 4 开始
任务 5 开始
任务 5 完成
任务 1 完成
任务 4 完成
任务 3 完成
任务 2 完成
线程池耗时: 2.04秒


#### 实例：数据读入   
数据准备

In [25]:
from glob import glob
import numpy as np
import time


In [26]:
for i in range(100):
    arr = np.random.rand(1000, 1000)  # 生成一个1000x1000的随机数组
    np.save(f'第7章-并发编程/data/data_{i}.npy', arr)  # 保存为.npy文件


In [27]:
paths = glob('第7章-并发编程/data/*.npy')  # 获取所有数据文件路径
# paths_list = [paths[i::2] for i in range(2)]
len(paths)


100

对比测试

In [28]:
def load_data(paths):
    data = []
    for path in paths:
        arr = np.load(path)  # 读取数据
        arr = arr+1
        data.append(arr)
    return len(data)


In [36]:
if __name__ == "__main__":
    # 测试单线程读取数据
    start = time.time()
    num_data = load_data(paths)  # 测试函数
    end = time.time()
    print(f'单线程读取数据个数: {num_data}')  # 输出读取的数据个数
    print(f'顺序读取耗时: {end - start:.2f}秒')  # 输出耗时


单线程读取数据个数: 100
顺序读取耗时: 1.25秒


In [40]:
if __name__ == "__main__":
    # 测试多线程读取数据
    paths_list = [paths[i::5] for i in range(5)]  # 将路径分成5份
    start = time.time()
    threads = []
    ### 扩展：推荐使用线程池
    for i in range(5):
        t = threading.Thread(target=load_data, name=f'Thread-{i+1}', args=(paths_list[i],))
        threads.append(t)
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    end = time.time()
    print(f'多线程读取耗时: {end - start:.2f}秒')


多线程读取耗时: 0.32秒
